In [4]:
import pandas as pd
import numpy as np
import os
import string

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff

import tensorflow as tf

In [5]:
file_name = "../data/drag-names.txt"

with open(file_name,'r') as f:
    names = f.read().split("\n")[:-1]

In [6]:
MAX_LENGTH = 31
names = [name for name in names if len(name)<=MAX_LENGTH]
print("Number of Names: ",len(names))

assert max(map(len,names))<=MAX_LENGTH, f"Names length more than {MAX_LENGTH}"

Number of Names:  3281


In [7]:
start_token = " "
pad_token = "#"

names = [start_token+name for name in names]
MAX_LENGTH += 1

tokens = sorted(set("".join(names + [pad_token])))

tokens = list(tokens)
n_tokens = len(tokens)

In [8]:
token_to_id = dict(zip(tokens,range(len(tokens))))

def to_matrix(names, max_len=None, pad=token_to_id[pad_token], dtype=np.int32):
    """Casts a list of names into rnn-digestable padded matrix"""

    max_len = max_len or max(map(len, names))
    names_ix = np.zeros([len(names), max_len], dtype) + pad

    for i in range(len(names)):
        name_ix = list(map(token_to_id.get, names[i]))
        names_ix[i, :len(name_ix)] = name_ix

    return names_ix

In [9]:
X = to_matrix(names)
X_train = np.zeros((X.shape[0],X.shape[1],n_tokens),np.int32)
y_train = np.zeros((X.shape[0],X.shape[1],n_tokens),np.int32)

for i, name in enumerate(X):
    for j in range(MAX_LENGTH-1):
        X_train[i,j,name[j]] = 1
        y_train[i,j,name[j+1]] = 1
    X_train[i,MAX_LENGTH-1,name[MAX_LENGTH-1]] = 1
    y_train[i,MAX_LENGTH-1,token_to_id[pad_token]] = 1

In [10]:
strategy = tf.distribute.get_strategy()

In [11]:
name_count = X.shape[0]

BATCH_SIZE = 64

STEPS_PER_EPOCH = np.ceil(name_count/BATCH_SIZE)

AUTO = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

In [12]:
train_dataset = (tf.data.Dataset.from_tensor_slices((X,y_train)).shuffle(5000).cache().repeat().batch(BATCH_SIZE).prefetch(AUTO))

In [13]:
num_rnn_units = 256
embedding_size = 16

def make_model():
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Embedding(n_tokens,embedding_size,input_length=MAX_LENGTH))

    model.add(tf.keras.layers.LSTM(num_rnn_units,return_sequences=True,activation='elu',input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(tf.keras.layers.LSTM(num_rnn_units,return_sequences=True,activation='elu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.LSTM(num_rnn_units,return_sequences=True,activation='elu'))
    
    model.add(tf.keras.layers.SimpleRNN(num_rnn_units,return_sequences=True,activation='elu'))
    model.add(tf.keras.layers.SimpleRNN(num_rnn_units,return_sequences=True,activation='elu'))
    model.add(tf.keras.layers.Dense(n_tokens,activation='softmax'))

    model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.001))

    return model

In [14]:
with strategy.scope():
    
    model = make_model()
    
    model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 16)            592       
                                                                 
 lstm (LSTM)                 (None, 32, 256)           279552    
                                                                 
 lstm_1 (LSTM)               (None, 32, 256)           525312    
                                                                 
 dropout (Dropout)           (None, 32, 256)           0         
                                                                 
 lstm_2 (LSTM)               (None, 32, 256)           525312    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32, 256)           131328    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32, 256)           1

In [15]:
class CyclicLR(tf.keras.callbacks.Callback):
    
    def __init__(self,base_lr=1e-5,max_lr=1e-3,stepsize=10):
        super().__init__()
        
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.stepsize = stepsize
        self.iterations = 0
        self.history = {}
        
    def clr(self):
        cycle = np.floor((1+self.iterations)/(2*self.stepsize))
        x = np.abs(self.iterations/self.stepsize - 2*cycle + 1)
        
        return self.base_lr + (self.max_lr - self.base_lr)*(np.maximum(0,1-x))
    
    def on_train_begin(self,logs={}):
        tf.keras.backend.set_value(self.model.optimizer.lr, self.base_lr)
    
    def on_batch_end(self,batch,logs=None):
        logs = logs or {}
        
        self.iterations += 1
        
        self.history.setdefault('lr', []).append(tf.keras.backend.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        tf.keras.backend.set_value(self.model.optimizer.lr, self.clr())
        

In [16]:
def generateName(model=model,seed_phrase=start_token,max_length=MAX_LENGTH):
    
    assert len(seed_phrase)<max_length, f"Length of the Seed-phrase is more than Max-Length: {max_length}"
    
    name = [seed_phrase]
    x = np.zeros((1,max_length),np.int32)

    x[0,0:len(seed_phrase)] = [token_to_id[token] for token in seed_phrase]
    
    for i in range(len(seed_phrase),max_length):
                
        probs = list(model.predict(x)[0,i-1])
        
        probs = probs/np.sum(probs)
        
        index = np.random.choice(range(n_tokens),p=probs)
        
        if index == token_to_id[pad_token]:
            break
            
        x[0,i] = index
        
        name.append(tokens[index])
    return "".join(name)

In [20]:
cyclicLR = CyclicLR(base_lr=1e-4,max_lr=1e-3,stepsize=6000)

EPOCHS = 100

history = model.fit(train_dataset,steps_per_epoch=STEPS_PER_EPOCH,epochs=EPOCHS,callbacks=[cyclicLR])

Epoch 1/100
52/52 [==============================] - 18s 275ms/step - loss: 1.9513
Epoch 2/100
52/52 [==============================] - 15s 293ms/step - loss: 1.5572
Epoch 3/100
52/52 [==============================] - 15s 290ms/step - loss: 1.5112
Epoch 4/100
52/52 [==============================] - 15s 295ms/step - loss: 1.4815
Epoch 5/100
52/52 [==============================] - 17s 334ms/step - loss: 1.4549
Epoch 6/100
52/52 [==============================] - 17s 318ms/step - loss: 1.4320
Epoch 7/100
52/52 [==============================] - 19s 362ms/step - loss: 1.3998
Epoch 8/100
52/52 [==============================] - 22s 424ms/step - loss: 1.3613
Epoch 9/100
52/52 [==============================] - 18s 343ms/step - loss: 1.3233
Epoch 10/100
52/52 [==============================] - 17s 325ms/step - loss: 1.2954
Epoch 11/100
52/52 [==============================] - 15s 294ms/step - loss: 1.2733
Epoch 12/100
52/52 [==============================] - 15s 294ms/step - loss: 1.2559
E

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(1,len(history.history['loss'])+1),
                        y=history.history['loss'],
                        mode='lines+markers',
                        name='Training loss'))

fig.update_layout(title_text="Training loss")

fig.show()

In [29]:
weights = 'weights2/DragNames(2SimpleRNN1000).h5'

In [30]:
model.save_weights(weights)

In [31]:
predictor = make_model()

In [32]:

predictor.load_weights(weights)

In [36]:

name = [' ']
x = np.zeros((1,32),np.int32)


In [48]:
probs = list(model.predict(x)[0,3])
    
probs = probs/np.sum(probs)
index = np.random.choice(range(n_tokens),p=probs)


18

In [49]:
n_tokens

37

In [33]:
seed_phrase = " "
for _ in range(100):
    name = generateName(predictor,seed_phrase=seed_phrase).split(" ")
    capitalized = [N.capitalize() for N in name]
    name = " ".join(capitalized)
    print(name.lstrip())

Bhitz
Madka Kalles
Sarlhas Miinmyslon
Krhalass Ke Phe
Carmew Virut
Esyy
Nani Stadr
Zabeon Fareri
Rilace Jagk
Mer Sciont
Kibbexnakte Ghar
Salsenin Mioen
Anba Petne
Lirao Saasi
Kalree Dofel
Avi Mame
Rogi E Clanes
Shenah Vorvape
Bechoene Ltiru
Jalua Lana
Gigen K Droer
Voosmaa Iranfono Andeora
Devin Hana
Anmeera Dsan Hapqaan
Duyli Lote
Jeara Pami
Loke Fend
Lad Birod
Amneny
Pavivlawi Avhuawberke
E
Sacema Jivinre
Pelldpie Nane Love
Lenh
Eypara Ganpo
Mene Mensar
Damparnota
Dunby Ke Luorers
Lagite Cormor
Taltya Stawhan
Lilia Durbel
Lonal A Conxlo
Chiodyt Pal
Kusty Mlaeto
Mubona Loi
Laxuxga Minre


KeyboardInterrupt: 